### Input files: Price-Today.csv

In [2]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

format_dict = {'qty':'{:,}','volbuy':'{:,}',
              'price_x':'{:,.2f}','price_y':'{:,.2f}','change':'{:,.2f}','price':'{:,.2f}',
              'chg_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'percent':'{:,.2f}','net':'{:,.2f}','dividend':'{:,.4f}'}

pd.options.display.max_rows = 31
pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-10-24 2025-10-23


In [3]:
# If run after actual work day
#today = today - timedelta(days=2)
#print(today)

In [4]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-10-24
yesterday: 2025-10-23


In [5]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-10-24 20:18:22


### Restart and Run All Cells

In [7]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


## Must rerun whenever there is change in orders

In [10]:
sql = """
SELECT name, date, volbuy, price, active, dividend, period, grade 
FROM buy
ORDER BY name"""
stocks = pd.read_sql(sql, const)
stocks['volbuy'] = stocks['volbuy'].astype('int64')
stocks.style.format(format_dict).hide(axis="index")

name,date,volbuy,price,active,dividend,period,grade
3BBIF,2018-05-17,"120,000",10.10,1,0.1546,3,A3
AH,2023-06-08,"1,200",37.00,1,0.7900,1,C1
AIMIRT,2023-08-17,"17,500",10.60,1,0.8500,2,A1
AWC,2023-06-15,"9,000",4.96,1,0.0750,1,A1
BCH,2021-09-07,"4,000",21.70,1,0.4300,3,B3
CPF,2025-08-21,"10,000",23.40,1,1.5500,4,A1
CPNREIT,2022-08-16,"55,000",18.00,1,0.7966,2,C1
DIF,2020-08-01,"45,000",12.70,1,0.8888,2,C1
GVREIT,2022-08-24,"69,000",7.75,1,0.8043,2,C1
IVL,2022-08-29,"7,200",40.00,1,0.7000,4,C3


In [11]:
names = stocks["name"].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'3BBIF', 'AH', 'AIMIRT', 'AWC', 'BCH', 'CPF', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'MCS', 'NER', 'ORI', 'PTG', 'PTT', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TOA', 'TVO', 'WHAIR', 'WHART'"

In [12]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [13]:
prices = pd.read_csv(input_file)
prices.shape

(168, 10)

In [14]:
df_merge  = pd.merge(stocks, prices, on = 'name', how='inner')
df_merge.shape

(29, 17)

### Convert column name to make it valid for df.query

In [16]:
pd.options.display.max_rows = 30
df_merge['percent'] = df_merge['%change']
df_merge['chg_amt'] = df_merge['volbuy'] * df_merge['change']
df_merge['volbuy'] = df_merge['volbuy'].astype('int64')
colo = ['name', 'volbuy', 'price_x', 'price_y', 'change', 'percent', 'chg_amt']
filter = df_merge['percent'] != 0
df_pct = df_merge[filter][colo].sort_values(by='percent', ascending=True).copy()
df_amt = df_merge[filter][colo].sort_values(by='chg_amt', ascending=True).copy()

In [17]:
df_pct.shape

(27, 7)

In [18]:
df_amt.shape

(27, 7)

In [19]:
df_pct.query('percent >= 3.00').style.format(format_dict).hide(axis="index")

name,volbuy,price_x,price_y,change,percent,chg_amt


In [20]:
df_pct.query('percent < -3.00').style.format(format_dict).hide(axis="index")

name,volbuy,price_x,price_y,change,percent,chg_amt


In [21]:
df_amt.query('chg_amt >= 6000').style.format(format_dict).hide(axis="index")

name,volbuy,price_x,price_y,change,percent,chg_amt


In [22]:
df_amt.query('chg_amt < -6000').style.format(format_dict).hide(axis="index")

name,volbuy,price_x,price_y,change,percent,chg_amt
RCL,"27,000",38.78,25.75,-0.25,-0.96,"-6,750.00"


In [23]:
float_formatter = "฿{:,.2f}"

In [24]:
float_formatter.format(df_amt.query('chg_amt >= 0')['chg_amt'].sum())

'฿21,975.00'

In [25]:
float_formatter.format(df_amt.query('chg_amt < 0')['chg_amt'].sum())

'฿-30,200.00'

In [26]:
float_formatter.format(df_amt.chg_amt.sum())

'฿-8,225.00'

In [27]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-10-24 20:18:23
